In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2024-01-01'
final_date = '2024-01-31'
query_error_year = 2024
query_error_month = 1

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2024\01. ENERO\DATA\ERROR ENERO 2024.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,858342a7-fd31-40a4-8cc1-8e360cb12698,FARMACIA FARMA FA,M289RR3NC,2022-11-06,1.80,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-11-07,11,2022,ERROR,ERROR_USUAL,858342a7-fd31-40a4-8cc1-8e360cb12698,2022-11-07,7,nan
1,1,da3a4b3a-7b06-49f1-8f59-ae60feececdc,El rincon del cantante,TNDPS2WWB,2022-12-14,0.95,ERROR,Bank account information not available,2022-12-14,12,2022,ERROR,SIN_CLABE,da3a4b3a-7b06-49f1-8f59-ae60feececdc,2022-12-14,14,nan
2,2,43685a7e-2583-480a-94be-f6aa7cfee8f6,NAILKERY SAN ANGELO,FYMY32LCT,2021-12-26,373.71,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2021-12-28,12,2021,ERROR,ERROR_USUAL,43685a7e-2583-480a-94be-f6aa7cfee8f6,2021-12-28,28,nan
3,3,3681aa22-8083-43a4-a552-ad7372081221,Bloombelle Lashes,RPB8JVFZD,2018-09-03,47.91,ERROR,failed swap,2018-09-03,9,2018,ERROR,ERROR_USUAL,3681aa22-8083-43a4-a552-ad7372081221,2018-09-03,3,nan
4,4,2fdcf827-4e1f-4b32-aa66-bdff89af941e,Bros Chicken,B484H8HYR,2021-02-21,362.21,ERROR,DEV.SPEICUENTA NO PERTE A BCO,2021-02-23,2,2021,ERROR,ERROR_USUAL,2fdcf827-4e1f-4b32-aa66-bdff89af941e,2021-02-23,23,nan


In [4]:
query_error.shape

(157709, 17)

In [5]:
query_error['mes_cierre'].unique()

array(['nan', 'Actual'], dtype=object)

In [6]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int32
year                             int32
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int32
mes_cierre                      object
dtype: object

In [7]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [8]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [9]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_15724\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
114059,114059,f19c69bb-2e8b-4688-949d-9cc6a0199829,NaN,A6QVBYZKZ,2023-12-31,"3,610.63",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2024-01-01,1,2024,ERROR,ERROR_USUAL,f19c69bb-2e8b-4688-949d-9cc6a0199829,2024-01-01,1,Actual
77419,77419,66e8c437-729d-4654-95c5-fd2e09a30d99,NaN,AXJRNPLSK,2023-12-31,"3,219.69",ERROR,DEV.SPEICUENTA INEXISTENTE,2024-01-01,1,2024,ERROR,ERROR_USUAL,66e8c437-729d-4654-95c5-fd2e09a30d99,2024-01-01,1,Actual
152146,152146,2c6a9f31-b80b-4a4d-978a-d983341e3a29,NaN,ATPX8HW8K,2024-01-01,348.80,ERROR,Bank account information not available,2024-01-01,1,2024,ERROR,SIN_CLABE,2c6a9f31-b80b-4a4d-978a-d983341e3a29,2024-01-01,1,Actual
138215,138215,1643e81b-db77-4da1-8c4c-04b78b6e87c7,NaN,ATMMDZD5T,2024-01-01,3.20,ERROR,Bank account information not available,2024-01-01,1,2024,ERROR,SIN_CLABE,1643e81b-db77-4da1-8c4c-04b78b6e87c7,2024-01-01,1,Actual
32647,32647,87b31f9f-4f79-4f48-9f49-852737d40646,NaN,AF68Q4LXH,2023-12-31,"2,364.76",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2024-01-01,1,2024,ERROR,ERROR_USUAL,87b31f9f-4f79-4f48-9f49-852737d40646,2024-01-01,1,Actual


In [10]:
query_error_current_month.shape

(10354, 17)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# AGRUPADO POR ESQUEMA

In [14]:
group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_15724\3966531006.py:1: FutureWarning: The provided callable <function sum at 0x000002CFFA9D5AF0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_15724\3966531006.py:1: FutureWarning: The provided callable <function sum at 0x000002CFFA9D5AF0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_15724\3966531006.py:1: FutureWarning: The provided callable <function sum at 0x000002CFFA9D5AF0> is currently us

ultimo_mov_date_only      amount                       \
esquema_query                      ERROR_USUAL            SIN_CLABE   
status_cubeta                            ERROR        NEW     ERROR   
0                       2024-01-01  180,970.61       0.00 88,170.59   
1                       2024-01-02   56,755.40 126,115.13      0.00   
2                       2024-01-03    5,772.67 145,426.65      0.00   
3                       2024-01-04   72,299.73  72,118.96      0.00   
4                       2024-01-05   50,333.20 111,439.14      0.00   

                                     
esquema_query                   All  
status_cubeta        NEW             
0                   0.00 269,141.20  
1              26,289.15 209,159.68  
2              70,118.22 221,317.54  
3             105,564.98 249,983.67  
4              91,584.51 253,356.85

In [15]:
group_esquema.shape

(32, 6)

In [16]:
writer = pd.ExcelWriter('FASE VI. DATA POLIZA ERROR-SIN CLABE ENERO 2024.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
query_error_current_month.to_excel(writer, sheet_name='DETALLE',index=False)
writer.close()